In [1]:
import os
 
from sqlalchemy import Column, DateTime, String, Integer, ForeignKey, func, create_engine
from sqlalchemy.orm import relationship, backref, sessionmaker
from sqlalchemy.ext.declarative import declarative_base

import pandas as pd
import requests

In [2]:
def get_user(i):
    users = []
    
    for j in range(i):
        user_data = requests.get('https://randomuser.me/api/').json()['results'][0]
        user = Employee(
            name = user_data['name']['first'] + ' '+ user_data['name']['last'],
            email = user_data['email'],
            country = user_data['location']['country'],
            city = user_data['location']['city'])
        
        comp = 'IT' if user.country in ['Netherlands', 'France', 'Germany'] else 'Estimating'
        company = session.query(Department).filter_by(name=comp).first() 
        
        
        link = DepartmentEmployeeLink(department=company, employee=user)

        users.append(link)
    
    return users

In [3]:
engine = create_engine('sqlite:///:memory:')

In [4]:
Base = declarative_base()

class Department(Base):
    __tablename__ = 'department'
    id = Column(Integer, primary_key=True) # autonumber
    name = Column(String)
    employees = relationship(
        'Employee',
        secondary='department_employee_link'
    )
 
 
class Employee(Base):
    __tablename__ = 'employee'
    id = Column(Integer, primary_key=True) # autonumber
    name = Column('name',String)
    email = Column('email',String)
    country = Column('country', String)
    city = Column('city', String)
    
    
    departments = relationship(
        Department,
        secondary='department_employee_link'
    )
 
 
class DepartmentEmployeeLink(Base):
    __tablename__ = 'department_employee_link'
    department_id = Column(Integer, ForeignKey('department.id'), primary_key=True)
    employee_id = Column(Integer, ForeignKey('employee.id'), primary_key=True)  
    
    department = relationship(Department, backref=backref("employee_assoc"))
    employee = relationship(Employee, backref=backref("department_assoc"))

In [5]:
Session = sessionmaker(bind=engine)
Base.metadata.create_all(bind=engine)
session = Session()

In [6]:
session =Session()

IT = Department(name = 'IT')
Estimating = Department(name = 'Estimating')

session.add_all([IT, Estimating])
session.commit()

U = get_user(20)

session.add_all(U)
session.commit()

In [7]:
pd.read_sql_table("department", engine)

,id,name
0,1,IT
1,2,Estimating


In [8]:
pd.read_sql_table("employee", engine)

,id,name,email,country,city
0,1,Louise Williamson,louise.williamson@example.com,United States,Spokane
1,2,Eddie Mckinney,eddie.mckinney@example.com,United States,Miramar
2,3,Jon Schmidt,jon.schmidt@example.com,Australia,Launceston
3,4,Frauke Steidle,frauke.steidle@example.com,Germany,Langenfeld (Rheinland)
4,5,Josep Blanco,josep.blanco@example.com,Spain,Vitoria
5,6,Marcus Lee,marcus.lee@example.com,New Zealand,Whangarei
6,7,Angie Fuller,angie.fuller@example.com,United Kingdom,Lichfield
7,8,Liam Ginnish,liam.ginnish@example.com,Canada,Jasper
8,9,مهراد حیدری,mhrd.hydry@example.com,Iran,مشهد
9,10,Beau Walker,beau.walker@example.com,New Zealand,Timaru


In [9]:
pd.read_sql_table("department_employee_link", engine)

,department_id,employee_id
0,2,1
1,2,2
2,2,3
3,1,4
4,2,5
5,2,6
6,2,7
7,2,8
8,2,9
9,2,10
